In [137]:
import pandas as pd

In [138]:
df = pd.read_csv('../data/dataJamal.csv')

In [139]:
df.head()

,NO,NAMA,NIK,ALAMAT,JUMLAH KELUARGA,PENGHASILAN,PENDIDIKAN TERTINGGI,SETATUS RUMAH,PEKERJAAN,KONDISI KESEHATAN,SETATUS PENERIMA
0,1,AI SITI NURSARIPAH,3213266101830001,KP CIJEREHILIR RW 003 RT 011,Keluarga Kecil,Sedang,SMP,Sedang,lain-lain,Cukup,Ya
1,2,AMANAH,3213265706680002,KP CIJEREHILIR RW 003 RT 007,Keluarga Sedang,Kurang,SMP,Sedang,petani,Cukup,Ya
2,3,ARWENAH,3213260705180000,KP CIGORE RW 005 RT 015,Keluarga Sedang,Tinggi,SARJANA,Baik,Wirausaha,Cukup,Tidak
3,4,AMNAH,3213264707420001,KMP CIJERE HILIR RW 003 RT 011,Keluarga Sedang,Sedang,SMP,Kurang,petani,Cukup,Ya
4,5,ANING,3213260909590004,KP CIJERE HILIR RW 003 RT 010,Keluarga Kecil,Kurang,SMP,Kurang,lain-lain,Cukup,Ya


In [140]:
import pandas as pd

def clean_data(df, drop_target_column=False):
    columns_to_drop = ['NAMA', 'NIK', 'ALAMAT']
    if drop_target_column:
        columns_to_drop.append('SETATUS PENERIMA')
    return df.drop(columns=columns_to_drop)

In [141]:
def transform_target(df, target_column):
    """
    Transformasi variabel target menjadi format numerik.
    
    Args:
    df (pd.DataFrame): DataFrame yang berisi data.
    target_column (str): Nama kolom target yang akan diubah.
    
    Returns:
    pd.DataFrame: DataFrame dengan kolom target yang telah diubah.
    """
    df[target_column] = df[target_column].apply(lambda x: 1 if x == 'Ya' else 0)
    return df


In [142]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

def encode_categorical_features(df, categorical_features):
    """
    Encode fitur kategori dengan OneHotEncoder.
    
    Args:
    df (pd.DataFrame): DataFrame yang berisi data.
    categorical_features (list): Daftar nama kolom kategori.
    
    Returns:
    pd.DataFrame: DataFrame dengan fitur kategori yang telah di-encode.
    """
    encoder = OneHotEncoder(drop='first', sparse_output=False)
    
    # Fit dan transformasi data kategori
    encoded_categories = encoder.fit_transform(df[categorical_features])
    
    # Nama-nama kolom hasil encoding
    feature_names = encoder.get_feature_names_out(categorical_features)
    
    # Gabungkan hasil encoding dengan DataFrame asli
    df_encoded = pd.DataFrame(encoded_categories, columns=feature_names)
    
    # Tambahkan kolom-kolom non-kategori yang tidak di-encode
    df_remaining = df.drop(columns=categorical_features)
    df_encoded = pd.concat([df_encoded, df_remaining.reset_index(drop=True)], axis=1)
    
    return df_encoded


In [143]:
from sklearn.preprocessing import StandardScaler

def standardize_numeric_features(df, numeric_features):
    """
    Standarisasi fitur numerik.
    
    Args:
    df (pd.DataFrame): DataFrame yang berisi data.
    numeric_features (list): Daftar nama kolom numerik.
    
    Returns:
    pd.DataFrame: DataFrame dengan fitur numerik yang telah distandarisasi.
    """
    scaler = StandardScaler()
    
    # Standarisasi fitur numerik
    df[numeric_features] = scaler.fit_transform(df[numeric_features])
    
    return df


In [144]:
from sklearn.model_selection import train_test_split

def split_data(df, target_column, test_size=0.2, random_state=42):
    """
    Pisahkan data menjadi set pelatihan dan pengujian.
    
    Args:
    df (pd.DataFrame): DataFrame yang berisi data.
    target_column (str): Nama kolom target.
    test_size (float): Proporsi data untuk set pengujian.
    random_state (int): Nilai random state untuk reprodusibilitas.
    
    Returns:
    tuple: (X_train, X_test, y_train, y_test)
    """
    X = df.drop(columns=[target_column])
    y = df[target_column]
    return train_test_split(X, y, test_size=test_size, random_state=random_state)


In [145]:
import pandas as pd


# Panggil fungsi preprocessing
data_cleaned = clean_data(df)
data_transformed = transform_target(data_cleaned, 'SETATUS PENERIMA')

categorical_features = ['JUMLAH KELUARGA', 'PENGHASILAN', 'PENDIDIKAN TERTINGGI', 'SETATUS RUMAH', 'PEKERJAAN', 'KONDISI KESEHATAN']

# Encode kategori dan standarisasi fitur numerik
data_encoded = encode_categorical_features(data_transformed, categorical_features)

# Kolom numerik yang mungkin ada, jika tidak ada bisa dilewati
numeric_features = []  # Kosongkan jika tidak ada fitur numerik

# Standarisasi jika ada fitur numerik
if numeric_features:
    data_final = standardize_numeric_features(data_encoded, numeric_features)
else:
    data_final = data_encoded

# Pisahkan data
X_train, X_test, y_train, y_test = split_data(data_final, 'SETATUS PENERIMA')

print("Data pelatihan dan pengujian telah dipersiapkan.")


Data pelatihan dan pengujian telah dipersiapkan.


In [146]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [147]:
# Inisialisasi dan latih model Naive Bayes
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [148]:
# Prediksi menggunakan data uji
y_pred = model.predict(X_test)


In [149]:
# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)


In [150]:
print("Akurasi model:", accuracy)
print("Laporan Klasifikasi:\n", report)

Akurasi model: 0.95
Laporan Klasifikasi:
               precision    recall  f1-score   support

           0       1.00      0.86      0.92        14
           1       0.93      1.00      0.96        26

    accuracy                           0.95        40
   macro avg       0.96      0.93      0.94        40
weighted avg       0.95      0.95      0.95        40



In [155]:
# Data Baru untuk Tes
new_data = pd.DataFrame({
    'NAMA': ['BU MARSHA', 'PAK RUDI'],
    'NIK': [3213266101830003, 3213265706680003],
    'ALAMAT': ['KP CIJEREHILIR RW 003 RT 012', 'KP CIJEREHILIR RW 003 RT 008'],
    'JUMLAH KELUARGA': ['Keluarga Sedang', 'Keluarga Kecil'],
    'PENGHASILAN PER BULAN (RP)': ['Tinggi', 'Sedang'],
    'PENDIDIKAN TERTINGGI': ['SMA', 'SD'],
    'SETATUS RUMAH': ['Baik', 'Sedang'],
    'PEKERJAAN': ['pedagang', 'buruh'],
    'KONDISI KESEHATAN': ['Baik', 'Kurang']
})

# Preprocessing data baru
# new_data_cleaned = clean_data(new_data)
# new_data_transformed = transform_target(new_data_cleaned, 'SETATUS PENERIMA')

# new_data_encoded = encode_categorical_features(new_data_transformed, categorical_features)
# new_data_final = standardize_numeric_features(new_data_encoded, numeric_features)

# Prediksi menggunakan model yang telah dilatih
new_predictions = model.predict(new_data)

# Tampilkan hasil prediksi
new_data['PREDIKSI SETATUS PENERIMA'] = ['Ya' if pred == 1 else 'Tidak' for pred in new_predictions]
print("Hasil Prediksi untuk Data Baru:\n", new_data)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- ALAMAT
- JUMLAH KELUARGA
- KONDISI KESEHATAN
- NAMA
- NIK
- ...
Feature names seen at fit time, yet now missing:
- JUMLAH KELUARGA_Keluarga Kecil
- JUMLAH KELUARGA_Keluarga Sedang
- KONDISI KESEHATAN_Cukup
- KONDISI KESEHATAN_Kurang
- NO
- ...
